##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Overview

This notebook demonstrates how to use Vertex AI Search for grounding Vertex LLMs.  For more general information on grounding, see [Getting Started with Grounding with Gemini in Vertex AI](./intro-grounding-gemini.ipynb).

In this tutorial, we will cover:

* How to create a Vertex AI Search datastore with your data.
* Provide an example LLM request that uses this data for grounding.

## Setup


### Import the Vertex AI Search API

In [ ]:
# Install the Client library
!pip install -U google-cloud-discoveryengine
!pip install -U google-cloud-aiplatform

## Authenticate

The Vertex AI Search API lets you perform both key word search and semantic search on your own data. Since it's **your data**, this needs stricter access controls than API keys. Authenticate with OAuth with [service accounts](#service-oauth).

This quickstart uses a simplified authentication approach meant for a testing environment, and service account setups are typically easier to start from. For a production environment, learn about [authentication and authorization](https://developers.google.com/workspace/guides/auth-overview){:.external} before choosing the [access credentials](https://developers.google.com/workspace/guides/create-credentials#choose_the_access_credential_that_is_right_for_you){:.external} that are appropriate for your app.


### Setup OAuth using service accounts {:#service-oauth}

Follow the steps below to setup OAuth using service accounts:


1. Enable the [Discovery Engine API](https://console.cloud.google.com/flows/enableapi?apiid=discoveryengine.googleapis.com):{.external} (a.k.a Vertex AI Search API).

<img width=400 src="https://ai.google.dev/tutorials/images/semantic_retriever_enable_api.png">

2. Create the Service Account by following the [documentation](https://developers.google.com/identity/protocols/oauth2/service-account#creatinganaccount).

 * After creating the service account, generate a service account key.
 * Grant `Discovery Engine Admin`, `GCE Storage Bucket Admin
`, Storage Admin IAM role to the service account.

<img width=400 src="https://ai.google.dev/tutorials/images/semantic_retriever_service_account.png">

3. Upload your service account file by using the file icon on the left sidebar, then the upload icon, as shown in the screenshot below.

 * Rename the uploaded file to `service_account_key.json` or change the variable `service_account_file_name` in the code below.

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

In [ ]:
!pip install -U google-auth-oauthlib

In [ ]:
service_account_file_name = 'cloud-ai-retail-search-test.json'

from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(service_account_file_name)

scoped_credentials = credentials.with_scopes(['https://www.googleapis.com/auth/cloud-platform'])

## Initialize client libarary

Initialize the client library using the service account credentials.

In [ ]:
# @markdown Replace TODO-your-project-name with your project name.
project = "cloud-ai-retail-search-test" #  @param {type:"string"}

# @markdown Select loction, could be global, us, eu.
location = 'global' # @param {type:"string"}

collection = 'default_collection'

# @markdown Set the engine id.
engine_id = 'grounding_demo_engine1' # @param {type: "string"}

# @markdown Set the datastore id.
data_store_id='grounding_demo_datastore2' # @param {type:"string"}

# @markdown Rename the uploaded file to `client_secret.json` Or Change the variable `client_file_name` below.
client_file_name = "cloud-ai-retail-search-test.json" # @param {type:"string"}

import google.cloud.discoveryengine_v1 as vais
import google.cloud.discoveryengine_v1alpha as vais_v1alpha
import google.api_core.client_options as gco
import google.api_core.operation as operation
import google.protobuf.any_pb2 as any_pb2
import time
import json
client_options = (
        gco.ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
data_store_service_client = vais.DataStoreServiceClient(client_options=client_options, credentials=scoped_credentials)
document_service_client = vais.DocumentServiceClient(client_options=client_options, credentials=scoped_credentials)

def wait_for_operation_finish(operation):
  while not operation.done:
    time.sleep(2) # sleep 2 seconds



## Create a DataStore {:#create-data-store}

In order to ingest your documents and perform searches, you need to create a datastore which means corpora.

Here we are creating a generic search data store with pdf documents.

In [ ]:
# Create a generic serch datastore with unstrucuted data content.
data_store = vais.DataStore(
    display_name='Grounding Demo DataStore',
    industry_vertical='GENERIC',
    solution_types=['SOLUTION_TYPE_SEARCH'],
    content_config='CONTENT_REQUIRED')

create_data_store_request = vais.CreateDataStoreRequest(
    parent=f'projects/{project}/locations/{location}/collections/{collection}',
    data_store=data_store,
    data_store_id=data_store_id
)

# The api returns long running operation as response.
create_data_store_operation = data_store_service_client.create_data_store(create_data_store_request)
print(f"Waiting for create datastore operation to complete: {create_data_store_operation.operation.name}")
created_data_store = create_data_store_operation.result()
data_store_name = created_data_store.name
print(f'Data store {data_store_name} is created.')


In [ ]:
print('Getting data store:')
get_data_store_request = vais.GetDataStoreRequest(name=data_store_name)
get_data_store_response = data_store_service_client.get_data_store(get_data_store_request)
print(get_data_store_response)

## Create a document

Upload your own documents to datastore

### Prepare the document in Google Cloud Storage

Vertex AI search supports variaty of data sources. Here it uses google cloud strage as the example.

In [ ]:
# install google cloud storage client library
!pip install -U google-cloud-storage

In [ ]:
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client(credentials=scoped_credentials)

# @markdown a GCS bucket used for VAIS document ingestion
bucket_name = "vais-grounding-demo-bucket1"  # @param {type:"string"}

# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name)

print(f"Bucket {bucket.name} created.")

In [ ]:
# Write the html docuements into gcs
from urllib.request import urlopen
import requests

file_urls = [
    "https://abc.xyz/assets/investor/static/pdf/2022_Q1_Earnings_Transcript.pdf",
    "https://abc.xyz/assets/investor/static/pdf/2022_Q2_Earnings_Transcript.pdf",
    "https://abc.xyz/assets/investor/static/pdf/2022_Q3_Earnings_Transcript.pdf",
    "https://abc.xyz/assets/investor/static/pdf/2022_Q4_Earnings_Transcript.pdf"
]

bucket = storage_client.bucket(bucket_name)

for url in file_urls:
    file_name = url.split("/")[-1]
    print(f"Downloading: {file_name}")

    try:
        response = requests.get(url)
        response.raise_for_status()

        # Construct the full blob path (including prefix)
        blob_name = f"{file_name}"
        blob = bucket.blob(blob_name)

        blob.upload_from_string(response.content)
        print(f"Uploaded: {blob_name}")  # Print the uploaded blob path
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {file_name}: {e}")


In [ ]:
# Check the crated files on GCS
for blob in bucket.list_blobs():
  print(blob.name)

### Upload documents to Datastore



In [ ]:
# Create document with gcs file.

created_document_names = []
id = 1
for url in file_urls:
    file_name = url.split("/")[-1]
    metadata = {
        'uri': url
    }

    document = vais.Document(
        content=vais.Document.Content(
            mime_type='application/pdf',
            uri=f'gs://{bucket_name}/{file_name}',
        ),
        # meta data
        json_data=json.dumps(metadata)
    )

    create_document_request = vais.CreateDocumentRequest(
        parent=f'{data_store_name}/branches/0',
        document_id=f'document-{id}',
        document=document
    )
    id += 1
    created_document = document_service_client.create_document(create_document_request)
    document_name = created_document.name
    created_document_names.append(document_name)
    print(f'Document {document_name} is created:')
    # print(created_document)



In [ ]:
import vertexai

from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
    grounding,
)

vertexai.init(project=project, location="us-central1",  credentials=scoped_credentials)

model = GenerativeModel("gemini-1.5-flash-001")

# The datastore,documents could have < 15m delay after creation.
tool = Tool.from_retrieval(
    grounding.Retrieval(
        grounding.VertexAISearch(
            datastore=data_store_id,
            project=project,
            location=location,
        )
    )
)

prompt = "What is the 2022 Q2 revenue?"
response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
    ),
)

print(response.text)

## Clean up

Use [`DeleteCorpusRequest`](https://ai.google.dev/api/python/google/generativeai/protos/DeleteCorpusRequest) to delete a user corpus and all associated `Document`s & `Chunk`s.

Note that non-empty corpora will throw an error without specifying an `force=True` flag. If you set `force=True`, any `Chunk`s and objects related to this `Document` will also be deleted.

If `force=False` (the default) and the `Document` contains any `Chunk`s, a `FAILED_PRECONDITION` error will be returned.

In [ ]:
# Delete the documents
for document_name in created_document_names:
  delete_document_request = vais.DeleteDocumentRequest(name=document_name)
  document_service_client.delete_document(delete_document_request)
  print(f'Successfully deleted docment: {document_name}')

In [ ]:
# Delete the data store
delete_data_store_request = vais.DeleteDataStoreRequest(name=data_store_name)
delete_data_store_operation = data_store_service_client.delete_data_store(delete_data_store_request)
print(f"Waiting for delete datastore operation to complete: {delete_data_store_operation.operation.name}")
wait_for_operation_finish(delete_data_store_operation)
print(f'Successfully deleted data store {data_store_name}')

## Appendix: Setup OAuth with user credentials {:#user-oauth}

Follow the steps below from the [OAuth Quickstart](https://ai.google.dev/docs/oauth_quickstart) to setup OAuth authentication.

1. [Configure the OAuth consent screen](https://ai.google.dev/docs/oauth_quickstart#configure-oauth).

1. [Authorize credentials for a desktop application](https://ai.google.dev/docs/oauth_quickstart#authorize-credentials). To run this notebook in Colab, first rename your credential file (usually `client_secret_*.json`) to just `client_secret.json`. Then upload the file by using the file icon on the left sidebar, then the upload icon, as shown in the screenshot below.

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

In [ ]:
# Replace TODO-your-project-name with the project used in the OAuth Quickstart
project_name = "TODO-your-project-name" #  @param {type:"string"}
# Replace TODO-your-email@gmail.com with the email added as a test user in the OAuth Quickstart
email = "TODO-your-email@gmail.com" #  @param {type:"string"}
# Rename the uploaded file to `client_secret.json` OR
# Change the variable `client_file_name` in the code below.
client_file_name = "client_secret.json"

# IMPORTANT: Follow the instructions from the output - you must copy the command
# to your terminal and copy the output after authentication back here.
!gcloud config set project $project_name
!gcloud config set account $email

# NOTE: The simplified project setup in this tutorial triggers a "Google hasn't verified this app." dialog.
# This is normal, click "Advanced" -> "Go to [app name] (unsafe)"
!gcloud auth application-default login --no-browser --client-id-file=$client_file_name --scopes="https://www.googleapis.com/auth/generative-language.retriever,https://www.googleapis.com/auth/cloud-platform"

Initialize the client library and re-run the notebook starting from [Create a corpus](#create-corpus).

In [ ]:
import google.ai.generativelanguage as glm

generative_service_client = glm.GenerativeServiceClient()
retriever_service_client = glm.RetrieverServiceClient()
permission_service_client = glm.PermissionServiceClient()